In [1]:
import sys
sys.path.append('../')

import csv
import torch
import numpy as np

from helpers import getEncoding
from torchnn import classArrayToLabel

In [2]:
w2v_vector_file = '../Data/Output/ct_report_unsupervised_W2V.csv'
lda_vector_file = '../Data/Output/ct_report_unsupervised_LDA.csv'
gt_file = '../Data/Output/ct_report_dmg_thoracic_only_LDAtrain_CHECKED_SAFE_COPY.csv'

In [3]:
# Loading ground truth
gt = {}

encoding=getEncoding(gt_file)
with open(gt_file, 'r', encoding=encoding) as gtin:
    gtreader = csv.reader(gtin, delimiter=',')
    for idx, c_cls, p_cls in gtreader:
        gt[idx] = {
            'C': c_cls,
            'P': p_cls
        }

encodeInfo:  {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [4]:
# Loading w2v data
w2v_data = {}

encoding=getEncoding(w2v_vector_file)
with open(w2v_vector_file, 'r', encoding=encoding) as w2vin:
    w2vreader = csv.reader(w2vin, delimiter=',')
    for idx, vec in w2vreader:
        if idx in gt:
            vec = vec.replace('[', '') \
                      .replace(']', '') \
                      .replace('\n', '')
            vec = np.fromstring(vec, sep=' ', dtype=np.double)
            w2v_data[idx] = vec

encodeInfo:  {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [5]:
# Loading w2v classifiers

c_classifier_file = 'C_CLASSIFIER_W2V'
p_classifier_file = 'P_CLASSIFIER_W2V'

In [6]:
nn_c = torch.load(c_classifier_file).double()
nn_p = torch.load(p_classifier_file).double()

In [7]:
# Classifying w2v data

correct_c = 0
correct_p = 0

for key, vector in w2v_data.items():
    vector = torch.tensor(vector, dtype=torch.double)
    c_pred = nn_c(vector)
    p_pred = nn_p(vector)

    c_label = classArrayToLabel(c_pred) + 'C'
    p_label = classArrayToLabel(p_pred) + 'P'

    correct_c += 1 if (gt[key]['C'] == c_label) else 0
    correct_p += 1 if (gt[key]['P'] == p_label) else 0

In [8]:
w2v_samples = len(w2v_data)

acc_w2v_c = 100 * correct_c / w2v_samples
acc_w2v_p = 100 * correct_p / w2v_samples

In [9]:
# Loading lda data
lda_data = {}

encoding=getEncoding(lda_vector_file)
with open(lda_vector_file, 'r', encoding=encoding) as ldain:
    ldareader = csv.reader(ldain, delimiter=',')
    for idx, vec in ldareader:
        if idx in gt:
            vec = vec.replace('[', '') \
                      .replace(']', '') \
                      .replace('\n', '')
            vec = np.fromstring(vec, sep=' ', dtype=np.double)
            lda_data[idx] = vec

encodeInfo:  {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [10]:
# Loading lda classifiers

c_classifier_file = 'C_CLASSIFIER_LDA'
p_classifier_file = 'P_CLASSIFIER_LDA'

nn_c = torch.load(c_classifier_file).double()
nn_p = torch.load(p_classifier_file).double()

In [11]:
# Classifying lda data

correct_c = 0
correct_p = 0

for key, vector in lda_data.items():
    vector = torch.tensor(vector, dtype=torch.double)
    c_pred = nn_c(vector)
    p_pred = nn_p(vector)

    c_label = classArrayToLabel(c_pred) + 'C'
    p_label = classArrayToLabel(p_pred) + 'P'

    correct_c += 1 if (gt[key]['C'] == c_label) else 0
    correct_p += 1 if (gt[key]['P'] == p_label) else 0

In [12]:
lda_samples = len(lda_data)

acc_lda_c = 100 * correct_c / lda_samples
acc_lda_p = 100 * correct_p / lda_samples

# Training accuracy

*NOTE: FLAIR training accuracy reported here are valuers reported by FLAIR and copied here from buildflair notebook.*

In [13]:
print('Training accuracy W2V for Cancer/No-Cancer  : %2.2f %%'%acc_w2v_c)
print('Training accuracy W2V for Prog  /No-Prog    : %2.2f %%'%acc_w2v_p)
print()
print('Training accuracy LDA for Cancer/No-Cancer  : %2.2f %%'%acc_lda_c)
print('Training accuracy LDA for Prog  /No-Prog    : %2.2f %%'%acc_lda_p)
print()
print('Training accuracy FLAIR for Cancer/No-Cancer: 73.05 %%')
print('Training accuracy FLAIR for Prog  /No-Prog  : 59.57 %%')

Training accuracy W2V for Cancer/No-Cancer  : 55.54 %
Training accuracy W2V for Prog  /No-Prog    : 50.50 %

Training accuracy LDA for Cancer/No-Cancer  : 44.46 %
Training accuracy LDA for Prog  /No-Prog    : 23.32 %

Training accuracy FLAIR for Cancer/No-Cancer: 73.05 %%
Training accuracy FLAIR for Prog  /No-Prog  : 59.57 %%
